### Codio Activity 4.2: Complex Joins on Datasets

**Expected Time: 60 Minutes**

**Total Points: 10**

![](images/kiva.png)

This assignment focuses on using pandas merge to answer questions using multiple data sources.  Here you will combine data and use many of the earlier `pandas` methods together with the `merge` function to extract insights from our data source.  The data comes from the Kiva loan provider organization and was part of a kaggle Data Science for good contest [here](https://www.kaggle.com/kiva/data-science-for-good-kiva-crowdfunding).

#### Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)
- [Problem 5](#Problem-5)

### The Data

Kiva allows users to fund loans to small organizations around the world.  The four datasets contain a subset of the data provided on kaggle and describe information on the individual loan, when it was given, where the borrowers were located, and what the loans were being used for.  Below, the data is loaded into four DataFrames and the first two rows of each displayed.  Note the shared `id` column across all four datasets.

In [1]:
import pandas as pd

/tmp/ipykernel_20160/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
crowdsource = pd.read_csv('data/kiva/crowdsource.csv')
crowdsource.head(2)

,id,posted_time,funded_time,lender_count
0,658776,2014-01-17 21:21:10+00:00,2014-02-05 17:57:55+00:00,33
1,1314847,2017-06-07 02:02:41+00:00,2017-06-21 17:10:38+00:00,9


In [3]:
demographics = pd.read_csv('data/kiva/demographics.csv')
demographics.head(2)

,id,country,region,borrower_genders
0,658776,El Salvador,Ciudad El Triunfo,male
1,1314847,Philippines,"Bais, Negros Oriental",female


In [4]:
financials = pd.read_csv('data/kiva/financials.csv')
financials.head(2)

,id,funded_amount,currency,term_in_months
0,658776,1000.0,USD,20.0
1,1314847,225.0,PHP,13.0


In [5]:
use = pd.read_csv('data/kiva/use.csv')
use.head(2)

,id,activity,sector,use
0,658776,Vehicle,Personal Use,to purchase a motorcycle in order to travel fr...
1,1314847,Pigs,Agriculture,to buy feed and other supplies like vitamins t...


[Back to top](#Index:) 

### Problem 1

#### Kenyan Loan Amounts

**5 Points**

Use the `demographics` and `financials` data to determine the average loan amount for the country Kenya.  Save your response as a float to `ans1` below.

In [12]:
### GRADED (Changed!)

ans1 = pd.merge(left=demographics, right=financials, on="id").query('country == "Kenya"')[["funded_amount"]].mean().values[0]

# Answer check
print(ans1)
print(type(ans1))

443.4723441615452
<class 'numpy.float64'>


[Back to top](#Index:) 

### Problem 2

### El Salvador Top Sector

**5 Points**

Use the `demographics` and `use` datasets to determine the sector of work that received the most loans in El Salvador.  Assign your result as a string to `ans2` below.

In [7]:
### GRADED
ans2 = pd.merge(left=demographics, right=use, on="id").query('country == "El Salvador"')["sector"].mode().item()
display(ans2)

# YOUR CODE HERE

# Answer check
print(ans2)
print(type(ans2))

'Agriculture'

Agriculture
<class 'str'>


### Problem 3

#### Pakistan Loans in Agriculture

**5 Points**

Combine the `use`, `demographics`, and `financials` data to determine the total amount of loans for Agriculture in Pakistan.  Assign your results as a float to `ans3` below.

In [14]:
### GRADED

ans3 = pd.merge(
    left=pd.merge(
        left=demographics,
        right=use,
        on="id"),
    right=financials,
    on="id").query('country == "Pakistan" and sector == "Agriculture"')[["funded_amount"]].sum().values[0]

# Answer check
print(ans3)
print(type(ans3))

30900.0
<class 'numpy.float64'>


[Back to top](#Index:) 

### Problem 4

#### Top Total Loan Sector

**5 Points**


What sector received the most total dollars in funding?  Assign your response as a string to `ans4` below.

In [9]:
### GRADED

ans4 = pd.merge(
    left=use,
    right=financials,
    on="id")[["sector", "funded_amount"]].groupby("sector").sum().idxmax().item()

# Answer check
print(ans4)
print(type(ans4))

Agriculture
<class 'str'>


[Back to top](#Index:) 

### Problem 5

#### Top Loan by Lender Amount

**5 Points**


Note that each loan is a combination of lenders.  Determine which loan sector has the highest ratio of dollar to lender amount.  Assign the sector with the highest ratio as a string to `ans5` below. 

In [10]:
### GRADED

temp_df = pd.merge(
    left=pd.merge(
        left=use,
        right=financials,
        on="id"),
    right=crowdsource,
    on="id")[["sector", "funded_amount", "lender_count"]]

#tt = temp_df.query("lender_count == 0")
#display(tt.shape)
#display(tt)
temp_df["dollar_to_lender_ratio"] = temp_df["funded_amount"]/temp_df["lender_count"]
temp_df = temp_df.dropna()
temp_df = temp_df.sort_values(by="dollar_to_lender_ratio", ascending=False)
display(temp_df)

ans5 = temp_df.iloc[0]["sector"]

# Answer check
print(type(ans5))
display(ans5)

,sector,funded_amount,lender_count,dollar_to_lender_ratio
9687,Retail,6000.0,1,6000.000000
1101,Retail,4850.0,1,4850.000000
6451,Clothing,8700.0,2,4350.000000
1030,Food,4100.0,1,4100.000000
2373,Arts,3650.0,1,3650.000000
...,...,...,...,...
1820,Clothing,125.0,16,7.812500
9927,Clothing,125.0,16,7.812500
5921,Food,125.0,16,7.812500
2850,Food,125.0,16,7.812500


<class 'str'>


'Retail'